In [ ]:
!pip install --upgrade -q git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier
!pip install --upgrade -q git+https://github.com/terrierteam/pyterrier_doc2query.git

In [1]:
import pyterrier as pt
pt.init()

PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [2]:
from pyterrier_doc2query import Doc2Query
doc2query = Doc2Query()

In [3]:
import pandas as pd
df = pd.DataFrame([
    {'docno': 'd1', 'text': "The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated"}
])
df

,docno,text
0,d1,The presence of communication amid scientific ...


In [4]:
doc2query(df)

,docno,text,querygen
0,d1,The presence of communication amid scientific ...,what effect did the manhattan project have on ...


In [5]:
dataset = pt.get_dataset("irds:vaswani")
index_loc = "./index_path"
indexer = Doc2Query(append=True, batch_size=20, num_samples=10) >> pt.IterDictIndexer(index_loc)
indexref = indexer.index(dataset.get_corpus_iter())

vaswani documents: 100%|██████████| 11429/11429 [18:57<00:00, 10.05it/s]


In [6]:
from pyterrier.measures import *
pt.Experiment([
    pt.BatchRetrieve.from_dataset('vaswani', wmodel="BM25"),
    pt.BatchRetrieve(indexref, wmodel="BM25"),
  ],
  dataset.get_topics(),
  dataset.get_qrels(),
  eval_metrics=[MAP, P@10, nDCG@10, Judged@10]
)

,name,AP,P@10,nDCG@10,Judged@10
0,BR(BM25),0.296517,0.352688,0.446609,0.351613
1,BR(BM25),0.299092,0.369892,0.460861,0.369892
